In [1]:
# importing packages
import pandas as pd
import numpy as np

In [2]:
# preprocessing the data
data = pd.read_csv('Resources/spam.csv', encoding = 'latin-1')

In [3]:
data.head()

,label,msg,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [4]:
data = data.drop(['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'], axis = 1)

In [5]:
# encoding Data
data['label'] = pd.factorize(data['label'])[0]

In [6]:
# module for removing unwanted words
import re
import nltk
nltk.download("stopwords")
from nltk.corpus import stopwords

# for stemming words
from nltk.stem.porter import PorterStemmer
temp = []

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/manohar/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [7]:
for row in data.itertuples():
    # to keep a - z letters and 0 - 9
    rev = re.sub("[^0-9a-zA-Z]"," ",row[2])
    rev = rev.lower()
    rev = rev.split()
    ps = PorterStemmer()
    rev = [ps.stem(word) for word in rev if not word in set(stopwords.words("english"))]
    rev = " ".join(rev)
    temp.append(rev)

In [8]:
data['msg'] = temp

In [10]:
# splitting data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data['msg'], data['label'], test_size = 0.25)

In [11]:
# creating bag of words model
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features = 410)
cv.fit(data['msg'])
X_train = cv.transform(X_train)
X_test = cv.transform(X_test)

In [16]:
# Training classifier
from sklearn import svm
classifier = svm.SVC()
classifier.fit(X_train, y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [14]:
y_pred = classifier.predict(X_test)

from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[1221    1]
 [  76   95]]


Using stacking

In [30]:
from sklearn.naive_bayes import GaussianNB
from sklearn.pipeline import make_pipeline, make_union
from tpot.builtins import StackingEstimator
from sklearn.linear_model import LogisticRegression

classifier = make_pipeline(StackingEstimator(estimator=GaussianNB()), LogisticRegression())
classifier.fit(X_train.toarray(), y_train)

Pipeline(memory=None,
     steps=[('stackingestimator', StackingEstimator(estimator=GaussianNB(priors=None))), ('logisticregression', LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False))])

In [31]:
y_pred = classifier.predict(X_test.toarray())

from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[1218    4]
 [  18  153]]
